In [ ]:
def compute_depth(x, y, img_left, img_right):
    """
    Computes the depth of the object in the camera frame.
    :param data: x, y, calibrated_left, calibrated_right
    :param x: centroid of the object, x coordinate
    :param y: centroid of the object, y coordinate
    :return: z (depth)
    """

    # GRAYSCALE
    gray_left = cv2.cvtColor(img_left, cv2.COLOR_BGR2GRAY)
    gray_right = cv2.cvtColor(img_right, cv2.COLOR_BGR2GRAY)

    # PARAMETERS
    min_disp = 0  # 22
    num_disp = 16*14  # 256
    block_size = 5  #5 
    sigma = 7 #1.5
    lmbda = 16000.0 #8000

    # DISPARITY MAP
    stereo_left = cv2.StereoBM_create(numDisparities=num_disp, blockSize=block_size)
    stereo_right = cv2.ximgproc.createRightMatcher(stereo_left)
    stereo_left.setMinDisparity(min_disp)
    stereo_left.setDisp12MaxDiff(1)  # 200
    stereo_left.setUniquenessRatio(1)  # 1
    stereo_left.setSpeckleRange(1)  # 10
    stereo_left.setSpeckleWindowSize(1)  # 3
    disp_left = stereo_left.compute(gray_left, gray_right)#.astype(np.float32)
    disp_left2 = cv2.normalize(disp_left, None, 255, 0, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    disp_right = stereo_right.compute(gray_right,gray_left)

    # Now create DisparityWLSFilter
    wls_filter = cv2.ximgproc.createDisparityWLSFilter(stereo_left)
    wls_filter.setLambda(lmbda)
    wls_filter.setSigmaColor(sigma)
    disp_filtered = wls_filter.filter(disp_left, gray_left, disparity_map_right=disp_right)
    disp_filtered[disp_filtered<-16] = -16
    disp_filtered = (disp_filtered+16)/5

    # Compute the Z coordinate
    Z = np.mean(disp_filtered[y-15:y+15,x-15:x+15]) # h,w format
    return Z
    
    